# Various small models for time series anomaly detection 

A number of simple models from some of the projects related to detecting anomalies in sensor data.

In [1]:
import os, sys
import pandas as pd
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), 'python'))
from nns.nns import ModelSummary, reset_keras, printable_dataframe, estimate
from nns import dlbs_models as models

from tensorflow.python.keras import layers, models, regularizers

#import tensorflow as tf
#import tensorflow.contrib.keras as K
import numpy as np

In [2]:
nfeatures = 41    # Number of sensors (features == time series)
window_size = 9   # Sliding window size for fully connected autoencoders
length = 100      # Input length for LSTM models (truncated backprop)

inference = []
training = []

# Fully-connected autoencoder

In [3]:
def fully_connected_autoencoder(nfeatures=41, window_size=9):
    """   369 -> 128 -> 64 -> 16 -> 64 -> 128 -> 369
    In some implementations decoder reconstructs only one window, for instance, in the center.
    """
    input_size = nfeatures * window_size

    inputs = layers.Input(shape=(input_size,), name='input')
    X = layers.Dense(128, activation='relu', name='enc/dense01')(inputs)
    X = layers.Dense(64, activation='relu', name='enc/dense02')(X)

    latent_repr = layers.Dense(16, activation='relu', activity_regularizer=regularizers.l1(10e-6),
                               name='enc/dense03')(X)

    X = layers.Dense(64, activation='relu', name='dec/dense01')(latent_repr)
    X = layers.Dense(128, activation='relu', name='dec/dense02')(X)
    outputs = layers.Dense(input_size, activation=None, name='dec/dense03')(X)

    model = models.Model(inputs, outputs, name="FullyConnectedAutoencoder")
    return model

In [4]:
estimate(fully_connected_autoencoder(nfeatures=nfeatures, window_size=window_size),
         inference,
         training)

Layer not recognized (type=<class 'tensorflow.python.keras.engine.input_layer.InputLayer'>, name=input)


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(369,)",0.000000,0,369,0.000000,0.001476
1,enc/dense01,"(128,)",0.000047,47360,256,0.189440,0.001024
2,enc/dense02,"(64,)",0.000008,8256,128,0.033024,0.000512
3,enc/dense03,"(16,)",0.000001,1040,32,0.004160,0.000128
4,dec/dense01,"(64,)",0.000001,1088,128,0.004352,0.000512
5,dec/dense02,"(128,)",0.000008,8320,256,0.033280,0.001024
6,dec/dense03,"(369,)",0.000047,47601,369,0.190404,0.001476
7,TOTAL,"(369,)",0.000113,113665,1538,0.454660,0.006152


# LSTM predictor

In [5]:
def lstm_predictor(nfeatures=41, length=100):
    inputs = layers.Input(shape=(length, nfeatures), name='input')
    X = layers.LSTM(128, return_sequences=True, unroll=True, name='lstm01')(inputs)
    X = layers.LSTM(128, return_sequences=False, unroll=True, name='lstm02')(X)
    outputs = layers.Dense(nfeatures, activation=None, name='output')(X)

    model = models.Model(inputs, outputs, name='LSTMPredictor')
    return model

In [6]:
estimate(lstm_predictor(nfeatures=nfeatures, length=length), inference, training)

W0522 16:12:04.717125 139664958748416 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f05a799ce48>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 16:12:13.166791 139664958748416 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f05a7b605f8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Layer not recognized (type=<class 'tensorflow.python.keras.engine.input_layer.InputLayer'>, name=input)


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(100, 41)",0.000000,0,4100,0.000000,0.016400
1,lstm01 (LSTM),"(100, 128)",0.008653,87040,179200,0.348160,0.716800
2,lstm02 (LSTM),"(128,)",0.013107,131584,179200,0.526336,0.716800
3,output,"(41,)",0.000005,5289,41,0.021156,0.000164
4,TOTAL,"(41,)",0.021765,223913,362541,0.895652,1.450164


# LSTM Autoencoder

In [7]:
def lstm_autoencoder(nfeatures=41, length=100, transfer_hs=False, version=1):
    # Build encoder except for the last layer (the one that computes code)
    inputs = layers.Input(shape=(length, nfeatures), name='input')
    X = layers.LSTM(64, return_sequences=True, unroll=True, name='enc/lsmt01')(inputs)
    X = layers.LSTM(32, return_sequences=True, unroll=True, name='enc/lsmt02')(X)
    
    # Build last encoder layer
    if transfer_hs is True:
        X, state_h, state_c = layers.LSTM(16, return_sequences=False, unroll=True, return_state=True,
                                          name='enc/lsmt03')(X)
        encoder_state = [state_h, state_c]
    else:
        X = layers.LSTM(16, return_sequences=False, unroll=True, name='enc/lsmt03')(X)
    X = layers.Dense(16, activation=None, name='enc/dense04')(X)

    if version == 1:
        # Repeat vector pattern
        X = layers.RepeatVector(length, name='enc/repeat')(X)
    elif version == 2:
        # Teacher forcing pattern
        # Concatenate along time dimension -> Batch, Length +1, features
        first_step = layers.Reshape((1, nfeatures), name='enc/reshape')(X)                # First decoder input is encoder output
        other_steps = layers.Lambda(lambda x: x[:, :-1, :], name='enc/lambda')(inputs)    # Everything else is input except last time
        X = layers.Concatenate(axis=1, name='enc/concatenate')([first_step, other_steps])
    
    # Build decoder
    if transfer_hs is True:
        X = layers.LSTM(16, return_sequences=True, unroll=True, name='dec/lstm01')(X, initial_state=encoder_state)
    else:
        X = layers.LSTM(16, return_sequences=True, unroll=True, name='dec/lstm01')(X)
    X = layers.LSTM(32, return_sequences=True, unroll=True, name='dec/lstm02')(X)
    X = layers.LSTM(64, return_sequences=True, unroll=True, name='dec/lstm03')(X)
    outputs = layers.TimeDistributed(layers.Dense(nfeatures, name='dec/dense04'), name='dec/dense_wrapper04')(X)
    
    model = models.Model(inputs, outputs, name='LSTMAutoencoder')
    model.compile(optimizer='sgd', loss='mse')
    return model

In [8]:
estimate(lstm_autoencoder(nfeatures=nfeatures, length=length, transfer_hs=False, version=1),
         inference,
         training)

W0522 16:13:00.827439 139664958748416 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f0415843f98>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 16:13:09.312294 139664958748416 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f0415878f98>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 16:13:17.468191 139664958748416 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f040446f780>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 16:13:25.638995 139664958748416 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f03f9c076a0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better perf

Layer not recognized (type=<class 'tensorflow.python.keras.engine.input_layer.InputLayer'>, name=input)


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(100, 41)",0.000000e+00,0,4100,0.000000,0.016400
1,enc/lsmt01 (LSTM),"(100, 64)",2.688000e-03,27136,89600,0.108544,0.358400
2,enc/lsmt02 (LSTM),"(100, 32)",1.228800e-03,12416,44800,0.049664,0.179200
3,enc/lsmt03 (LSTM),"(16,)",3.072000e-04,3136,22400,0.012544,0.089600
4,enc/dense04,"(16,)",2.560000e-07,272,16,0.001088,0.000064
5,enc/repeat,"(100, 16)",0.000000e+00,0,1600,0.000000,0.006400
6,dec/lstm01 (LSTM),"(100, 16)",2.048000e-04,2112,22400,0.008448,0.089600
7,dec/lstm02 (LSTM),"(100, 32)",6.144000e-04,6272,44800,0.025088,0.179200
8,dec/lstm03 (LSTM),"(100, 64)",2.457600e-03,24832,89600,0.099328,0.358400
9,dec/dense04,"(41,)",2.624000e-04,2665,4100,0.010660,0.016400


### Summary: inference

In [9]:
printable_dataframe(inference)

,Model,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,FullyConnectedAutoencoder,"(369,)",113665,0.454660,0.000113,0.006152
1,LSTMPredictor,"(100, 41)",223913,0.895652,0.021765,1.450164
2,LSTMAutoencoder,"(100, 41)",78841,0.315364,0.007763,1.293664


### Summary: training

In [10]:
printable_dataframe(training)

,Model,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,FullyConnectedAutoencoder,"(369,)",113665,0.454660,0.000339,0.012304
1,LSTMPredictor,"(100, 41)",223913,0.895652,0.065296,2.900328
2,LSTMAutoencoder,"(100, 41)",78841,0.315364,0.023290,2.587328
